In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 心得
- 滿快就寫完的，簡單測驗multimodal vs pure。

In [ ]:
pre_dir = '/kaggle/input/Kannada-MNIST/'

train_data = pd.read_csv(pre_dir + 'train.csv')
test_data = pd.read_csv(pre_dir + 'test.csv')

train_data.head()

In [ ]:
test_data.head()

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
# 了解pixel

train_data.describe()

In [ ]:
# 需要將資料標準化，但要用z-score / minmax呢?
# minmax適合均勻分配的資料, z-score適合標準常態
# 但對於圖片資料來說，是視為一個整體，而不是一個column之間是獨立的，故minmax較合適，

train_data.iloc[:, 1:] = train_data.iloc[:, 1:] / 255.0
test_data.iloc[:, 1:] = test_data.iloc[:, 1:] / 255.0

In [ ]:
import tensorflow as tf
from tensorflow import keras

tf.__version__

In [ ]:
# # 資料處理完畢就進入模型: 此處只要檢驗multimodal pattern
# # 法一: 單純CNN結構(圖片輸入)
# # 法二: CNN + tabular features


# # 法一: 將資料轉成28, 28, 1

# def get_pure_cnn_model():
#     """
#         透過Functional API拼接。
#     """
#     inputs = keras.Input(shape=(784, ))
#     x = keras.layers.Reshape((28, 28, 1))(inputs)
#     x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(x)
#     x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(x)  
#     x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(x)
#     x = keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)
#     x = keras.layers.Flatten()(x)
#     outputs = keras.layers.Dense(10, activation='softmax')(x)
    
#     model = keras.Model(inputs, outputs)
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
#     return model

# pure_cnn = get_pure_cnn_model()
# pure_cnn.fit(train_data.iloc[:, 1:], train_data.iloc[:, 0], validation_split=0.2, batch_size=128, epochs=10)

In [ ]:
# # result for pure cnn
# pred_pure_cnn = pure_cnn.predict(test_data.iloc[:, 1:])
# pred_pure_cnn = np.argmax(pred_pure_cnn, axis=1)
# pred_pure_cnn = np.concatenate([test_data.iloc[:, 0:1], pred_pure_cnn.reshape(-1, 1)], axis=1)
# pred_pure_cnn = pd.DataFrame(pred_pure_cnn, columns=['id', 'label'])
# pred_pure_cnn.to_csv('submission.csv', index=False)

In [ ]:
# pred_pure_cnn

In [ ]:
def get_multimodal_model():
    """
        透過Functional API拼接。
    """
    inputs = keras.Input(shape=(784, ))
    x = keras.layers.Reshape((28, 28, 1))(inputs)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(x)  
    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Concatenate()([inputs, x])
    outputs = keras.layers.Dense(10, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
    return model

ultimodal_cnn = get_multimodal_model()
ultimodal_cnn.fit(train_data.iloc[:, 1:], train_data.iloc[:, 0], validation_split=0.2, batch_size=128, epochs=10)

In [ ]:
# result for multimodal cnn
pred_ultimodal_cnn = ultimodal_cnn.predict(test_data.iloc[:, 1:])
pred_ultimodal_cnn = np.argmax(pred_ultimodal_cnn, axis=1)
pred_ultimodal_cnn = np.concatenate([test_data.iloc[:, 0:1], pred_ultimodal_cnn.reshape(-1, 1)], axis=1)
pred_ultimodal_cnn = pd.DataFrame(pred_ultimodal_cnn, columns=['id', 'label'])
pred_ultimodal_cnn.to_csv('submission.csv', index=False)

In [ ]:
pred_ultimodal_cnn

## 結果
- pure
- multimodal